In [28]:
import numpy as np
import matplotlib.pyplot as plt

from urllib.request import urlopen
import re
from bs4 import BeautifulSoup

import openai

# First, we want to collect a list of URLs that point to all housing postings

In [29]:
# First, set up all urls to try scraping
page_urls_to_try = []
page_urls_to_try.append( "https://sites.dartmouth.edu/dartlist/category/housing/" )
for j in range(2,13):
    page_urls_to_try.append(f"https://sites.dartmouth.edu/dartlist/category/housing/page/{j}")

# Second, create master list of all links to posts
all_links = []

# Third, loop over all page urls we are trying and scrape all links
for j, page_url_to_try in enumerate(page_urls_to_try):
    try:
        # Open the initial webpage
        url = page_url_to_try
        page = urlopen(url)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")

        # Collect all post elements
        posts_on_page = soup.find_all("div", {"class": "post-container"})

        # Loop over all posts and add to list of all posts
        for post in posts_on_page:
            all_links.append( post.find("h2").a["href"] )

        print(f"Post links collected for page {j+1}")
    except:
        pass


Post links collected for page 1
Post links collected for page 2
Post links collected for page 3
Post links collected for page 4
Post links collected for page 5
Post links collected for page 6
Post links collected for page 7
Post links collected for page 8
Post links collected for page 10
Post links collected for page 11


In [43]:
print(all_links)

['https://sites.dartmouth.edu/dartlist/2023/05/02/searching-for-apartment-in-hanover-and-or-roomates-to-split-lease-with/', 'https://sites.dartmouth.edu/dartlist/2023/05/02/sublet-available-in-downtown-hanover-for-june/', 'https://sites.dartmouth.edu/dartlist/2023/05/02/single-bedroom-available-for-1-year-lease-from-july-31st-2023-3-minute-walk-from-the-green/', 'https://sites.dartmouth.edu/dartlist/2023/05/01/950-month-summer-sublet-6-1-7-31-1-bedroom-in-white-river-junction/', 'https://sites.dartmouth.edu/dartlist/2023/05/01/looking-for-short-term-sublease-in-hanover-6-11-7-8/', 'https://sites.dartmouth.edu/dartlist/2023/05/01/looking-for-roommates-for-newly-remodeled-4bed-2bath-house-in-hanover-west-lebanon/', 'https://sites.dartmouth.edu/dartlist/2023/05/01/looking-for-female-roommate-for-summit-on-juniper/', 'https://sites.dartmouth.edu/dartlist/2023/05/01/looking-for-2-roommates-to-share-my-furnished-apartment-with-in-downtown-lebanon-at-an-affordable-price/', 'https://sites.dart

# Now we have all links, we will go through each one and construct a database

In [31]:
# Open the post
url = all_links[0]
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")

In [32]:
# Collect all post elements
post_content = soup.find("div", {"class": "post-content"})

In [35]:
post_content.p.get_text()

'Hi, I’m currently searching for an APARTMENT in Hanover, not a sublet or room. However, I’m open to splitting a lease with someone I met first, that way more options open up affordability wise. I’m looking to move immediately, and pretty flexible when it comes to the length of the lease. Again, when my price range extends hopefully more options to choose from will open up. '

In [ ]:
openai.api_key = ''

In [42]:
for j, link in enumerate(all_links[:30]):

    print( )
    print( )
    print(f"Checking out post #{j+1}")
    print(link)

    # Open the post
    url = link
    page = urlopen(url)
    html = page.read().decode("utf-8")
    soup = BeautifulSoup(html, "html.parser")
    
    try: 
        # Collect all post elements
        post_content = soup.find("div", {"class": "post-content"})
        post_text = post_content.p.get_text()


        try:

            prompt = "Read the following rental advertisement, and tell me what the per month price is. Give your answer as a single dollar amount, with no other explanation."
            prompt = prompt + " " + post_text

            completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                                                    messages=[ {"role": "user", 
                                                                "content": prompt
                                                            }])
            
            reply = completion.choices[0].message.content
            print(reply)

            prompt = """Read the following rental advertisement, and tell me what city the apartment is located in. Give your answer as "City, State" with no other explanation. If you cannot determine what city, simply state "N/A"."""
            prompt = prompt + " " + post_text

            completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", 
                                                    messages=[ {"role": "user", 
                                                                "content": prompt
                                                            }])

            reply = completion.choices[0].message.content

            print(reply)

        except:
            pass

    except:
        pass



Checking out post #1
https://sites.dartmouth.edu/dartlist/2023/05/02/searching-for-apartment-in-hanover-and-or-roomates-to-split-lease-with/
Unknown. The advertisement does not provide information about the monthly price.
Hanover, N/A


Checking out post #2
https://sites.dartmouth.edu/dartlist/2023/05/02/sublet-available-in-downtown-hanover-for-june/
I'm sorry, I cannot provide an answer as there is no rental advertisement provided in the prompt.
N/A


Checking out post #3
https://sites.dartmouth.edu/dartlist/2023/05/02/single-bedroom-available-for-1-year-lease-from-july-31st-2023-3-minute-walk-from-the-green/
$600
Hanover, NH


Checking out post #4
https://sites.dartmouth.edu/dartlist/2023/05/01/950-month-summer-sublet-6-1-7-31-1-bedroom-in-white-river-junction/
$950
Norwich, Vermont


Checking out post #5
https://sites.dartmouth.edu/dartlist/2023/05/01/looking-for-short-term-sublease-in-hanover-6-11-7-8/
Unfortunately, the rental advertisement does not state the per month price.
Ha